In [1]:
# libraries
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
import gensim
import sklearn
import re, os, string

In [2]:
print 'Punctuation - ',string.punctuation
stop_words = set(stopwords.words('english'))
print 'Stopwords - ', stop_words

Punctuation -  !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
Stopwords -  set([u'all', u'just', u"don't", u'being', u'over', u'both', u'through', u'yourselves', u'its', u'before', u'o', u'don', u'hadn', u'herself', u'll', u'had', u'should', u'to', u'only', u'won', u'under', u'ours', u'has', u"should've", u"haven't", u'do', u'them', u'his', u'very', u"you've", u'they', u'not', u'during', u'now', u'him', u'nor', u"wasn't", u'd', u'did', u'didn', u'this', u'she', u'each', u'further', u"won't", u'where', u"mustn't", u"isn't", u'few', u'because', u"you'd", u'doing', u'some', u'hasn', u"hasn't", u'are', u'our', u'ourselves', u'out', u'what', u'for', u"needn't", u'below', u're', u'does', u"shouldn't", u'above', u'between', u'mustn', u't', u'be', u'we', u'who', u"mightn't", u"doesn't", u'were', u'here', u'shouldn', u'hers', u"aren't", u'by', u'on', u'about', u'couldn', u'of', u"wouldn't", u'against', u's', u'isn', u'or', u'own', u'into', u'yourself', u'down', u"hadn't", u'mightn', u"couldn't", u'wasn', u'y

In [3]:
text = open('Medline/text_files/doc1').read()
print 'Doc1 - \n',text

Doc1 - 
correlation between maternal and fetal plasma levels of glucose and free
fatty acids .                                                           
  correlation coefficients have been determined between the levels of   
glucose and ffa in maternal and fetal plasma collected at delivery .    
significant correlations were obtained between the maternal and fetal   
glucose levels and the maternal and fetal ffa levels . from the size of 
the correlation coefficients and the slopes of regression lines it      
appears that the fetal plasma glucose level at delivery is very strongly
dependent upon the maternal level whereas the fetal ffa level at        
delivery is only slightly dependent upon the maternal level .           


In [4]:
doc = re.sub('\r\n|\r','',text)

In [5]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

tokens = []

for token in nltk.word_tokenize(doc):
    if token not in string.punctuation and token not in stop_words:
        tokens.append(token)

print 'Comparing stemming and lemmatization\nToken\t\tStemmed Token\t\tLemmatized Token'
for token in list(set(tokens)):
    print token,'\t\t',stemmer.stem(token),'\t\t',lemmatizer.lemmatize(token)
    
print '\nLemmatization it is!'

Comparing stemming and lemmatization
Token		Stemmed Token		Lemmatized Token
correlations 		correl 		correlation
regression 		regress 		regression
ffa 		ffa 		ffa
size 		size 		size
whereas 		wherea 		whereas
collected 		collect 		collected
slopes 		slope 		slope
plasma 		plasma 		plasma
glucose 		glucos 		glucose
maternal 		matern 		maternal
dependent 		depend 		dependent
upon 		upon 		upon
delivery 		deliveri 		delivery
levels 		level 		level
significant 		signific 		significant
freefatty 		freefatti 		freefatty
appears 		appear 		appears
coefficients 		coeffici 		coefficient
obtained 		obtain 		obtained
level 		level 		level
stronglydependent 		stronglydepend 		stronglydependent
acids 		acid 		acid
lines 		line 		line
determined 		determin 		determined
correlation 		correl 		correlation
slightly 		slightli 		slightly
fetal 		fetal 		fetal

Lemmatization it is!


In [6]:
# Loading docs
path = 'Medline/text_files'
docs = {}
for subdir, dirs, files in os.walk(path):
    for file in files:
        file_path = subdir + os.path.sep + file
        doc = re.sub('\r\n|\r',' ',open(file_path, 'r').read())
        docs[file] = doc

In [7]:
docs = [ (int(re.search('[0-9]+',x).group(0)), y) for x,y in docs.items()]
docs[:2]

[(365,
  '2469. the nature of collagen disease, particularly of systemic lupus erythematosus (sle), with special reference to renal lesions (japanese) from the results of examination of 161 autopsy cases of collagen diseases, rheumatic fever and polyarteritis nodosa (pn) are considered as diseases of hyperergic nature, showing pronounced specific manifestation in particular organs.  as the reactivity becomes lower, this specific localization becomes less significant and many organs become affected in sle.  it may accordingly be supposed that sle is not a disease of hypersensitivity but one occurring in a state of exhaustion of reactability after prolonged sensitization.  moreover, a sle-like syndrome arises not only in the end stage of parasepsis, nephritis, nephrosis and some cases of prolonged sensitization by myco.tb but also in pn,pss,dm and rheumatism.  it may therefore be conceivable that sle is a syndrome rather than a separate entity.  it is most important to consider whether s

In [8]:
docs.sort(key = lambda x: x[0])
docs[:2]

[(0, ''),
 (1,
  'correlation between maternal and fetal plasma levels of glucose and free fatty acids .                                                              correlation coefficients have been determined between the levels of    glucose and ffa in maternal and fetal plasma collected at delivery .     significant correlations were obtained between the maternal and fetal    glucose levels and the maternal and fetal ffa levels . from the size of  the correlation coefficients and the slopes of regression lines it       appears that the fetal plasma glucose level at delivery is very strongly dependent upon the maternal level whereas the fetal ffa level at         delivery is only slightly dependent upon the maternal level .            ')]

In [9]:
docs = docs[1:]
docs = [y for x,y in docs]
docs[:2]

['correlation between maternal and fetal plasma levels of glucose and free fatty acids .                                                              correlation coefficients have been determined between the levels of    glucose and ffa in maternal and fetal plasma collected at delivery .     significant correlations were obtained between the maternal and fetal    glucose levels and the maternal and fetal ffa levels . from the size of  the correlation coefficients and the slopes of regression lines it       appears that the fetal plasma glucose level at delivery is very strongly dependent upon the maternal level whereas the fetal ffa level at         delivery is only slightly dependent upon the maternal level .            ',
 'changes of the nucleic acid and phospholipid levels of the livers in the course of fetal and postnatal development .                                we have followed the evolution of dna, rna and pl in the livers of rat foeti removed between the fifteenth and the 

In [10]:
# tokenize function
def tokenize(text):
    tokens = []
    for token in nltk.word_tokenize(text.lower()):
        if token not in string.punctuation and token not in stop_words:
            tokens.append(lemmatizer.lemmatize(token))
    tokens = [token for token in tokens if token.isalpha() and len(token)>2]
    return tokens   

In [11]:
import nltk
doc_tokens = [list(tokenize(doc_text)) for doc_text in docs]
doc_tokens[:2]

[['correlation',
  'maternal',
  'fetal',
  'plasma',
  u'level',
  'glucose',
  'free',
  'fatty',
  u'acid',
  'correlation',
  u'coefficient',
  'determined',
  u'level',
  'glucose',
  'ffa',
  'maternal',
  'fetal',
  'plasma',
  'collected',
  'delivery',
  'significant',
  u'correlation',
  'obtained',
  'maternal',
  'fetal',
  'glucose',
  u'level',
  'maternal',
  'fetal',
  'ffa',
  u'level',
  'size',
  'correlation',
  u'coefficient',
  u'slope',
  'regression',
  u'line',
  'appears',
  'fetal',
  'plasma',
  'glucose',
  'level',
  'delivery',
  'strongly',
  'dependent',
  'upon',
  'maternal',
  'level',
  'whereas',
  'fetal',
  'ffa',
  'level',
  'delivery',
  'slightly',
  'dependent',
  'upon',
  'maternal',
  'level'],
 [u'change',
  'nucleic',
  'acid',
  'phospholipid',
  u'level',
  u'liver',
  'course',
  'fetal',
  'postnatal',
  'development',
  'followed',
  'evolution',
  'dna',
  'rna',
  u'liver',
  'rat',
  'foeti',
  'removed',
  'fifteenth',
  'day',

In [12]:
# tf-idf matrix - for clustering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

tfidf = TfidfVectorizer(tokenizer=tokenize)
tfs = tfidf.fit_transform(docs)

counter = CountVectorizer(tokenizer=tokenize)
counter_ = counter.fit_transform(docs)



In [25]:
import json
json.dump(counter.vocabulary_, open('json_files/index_terms','w'))

In [21]:
counter.vocabulary_

{u'diffusely': 2502,
 u'cussed': 2118,
 u'glisson': 3826,
 u'yellow': 10281,
 u'four': 3637,
 u'supposes': 9024,
 u'localized': 5326,
 u'biomorphosis': 997,
 u'neuroeffectors': 6113,
 u'albumen': 296,
 u'immature': 4434,
 u'kwashiorkor': 5074,
 u'dissolution': 2616,
 u'cytochrome': 2143,
 u'hormone': 4216,
 u'shielding': 8451,
 u'cardiomegaly': 1222,
 u'regional': 7826,
 u'pigment': 6936,
 u'leukon': 5216,
 u'leisurely': 5178,
 u'hydrolyzate': 4259,
 u'experimentally': 3276,
 u'bringing': 1088,
 u'polyacrylamide': 7046,
 u'differentiated': 2492,
 u'wooden': 10243,
 u'persisted': 6823,
 u'liation': 5227,
 u'intensification': 4757,
 u'provided': 7457,
 u'pitressin': 6953,
 u'reliable': 7881,
 u'fect': 3421,
 u'initiating': 4678,
 u'specially': 8650,
 u'iliacus': 4412,
 u'cytomegalovirus': 2153,
 u'pubic': 7510,
 u'pulse': 7525,
 u'second': 8290,
 u'sustaining': 9073,
 u'bilharziasis': 969,
 u'relieving': 7887,
 u'nonfunctioning': 6204,
 u'binasal': 979,
 u'contributed': 1897,
 u'postoper

In [13]:
len(counter.vocabulary_)

10306

In [15]:
tfidf.vocabulary_

{u'diffusely': 2502,
 u'cussed': 2118,
 u'glisson': 3826,
 u'yellow': 10281,
 u'four': 3637,
 u'supposes': 9024,
 u'localized': 5326,
 u'biomorphosis': 997,
 u'neuroeffectors': 6113,
 u'albumen': 296,
 u'immature': 4434,
 u'kwashiorkor': 5074,
 u'dissolution': 2616,
 u'cytochrome': 2143,
 u'hormone': 4216,
 u'shielding': 8451,
 u'cardiomegaly': 1222,
 u'regional': 7826,
 u'pigment': 6936,
 u'leukon': 5216,
 u'leisurely': 5178,
 u'hydrolyzate': 4259,
 u'experimentally': 3276,
 u'bringing': 1088,
 u'polyacrylamide': 7046,
 u'differentiated': 2492,
 u'wooden': 10243,
 u'persisted': 6823,
 u'liation': 5227,
 u'intensification': 4757,
 u'provided': 7457,
 u'pitressin': 6953,
 u'reliable': 7881,
 u'fect': 3421,
 u'initiating': 4678,
 u'specially': 8650,
 u'iliacus': 4412,
 u'cytomegalovirus': 2153,
 u'pubic': 7510,
 u'pulse': 7525,
 u'second': 8290,
 u'sustaining': 9073,
 u'bilharziasis': 969,
 u'relieving': 7887,
 u'nonfunctioning': 6204,
 u'binasal': 979,
 u'contributed': 1897,
 u'postoper

In [16]:
tfidf.vocabulary_.keys

<function keys>

In [17]:
print tfs

  (0, 1959)	0.2676401643093625
  (0, 5570)	0.45561141415090967
  (0, 3452)	0.45111651060993085
  (0, 6975)	0.17076170229568946
  (0, 5221)	0.36499301658324745
  (0, 3842)	0.27682412904155307
  (0, 3654)	0.0645431280994378
  (0, 3405)	0.06778768230257375
  (0, 107)	0.050769192966264996
  (0, 1580)	0.17315638538807124
  (0, 2412)	0.05985413295296133
  (0, 3458)	0.2761643185408719
  (0, 1618)	0.09000445956572757
  (0, 2294)	0.24105738585285244
  (0, 8494)	0.050620838756254644
  (0, 6319)	0.050918994557271606
  (0, 8548)	0.064913134950086
  (0, 8569)	0.09000445956572757
  (0, 7834)	0.07755299807955797
  (0, 5263)	0.06970950494873349
  (0, 582)	0.06012590184030526
  (0, 8865)	0.08254970976290668
  (0, 2340)	0.1628236316894799
  (0, 9868)	0.12435243024307
  (0, 10190)	0.06778768230257375
  :	:
  (1032, 8420)	0.05849411671137939
  (1032, 5350)	0.0744480325541736
  (1032, 4097)	0.06889019105555322
  (1032, 1132)	0.0713180185409884
  (1032, 2632)	0.0713180185409884
  (1032, 634)	0.0713180185409

In [18]:
print counter_

  (0, 8568)	1
  (0, 10190)	1
  (0, 9868)	2
  (0, 2340)	2
  (0, 8865)	1
  (0, 582)	1
  (0, 5263)	1
  (0, 7834)	1
  (0, 8569)	1
  (0, 8548)	1
  (0, 6319)	1
  (0, 8494)	1
  (0, 2294)	3
  (0, 1618)	1
  (0, 3458)	3
  (0, 2412)	1
  (0, 1580)	2
  (0, 107)	1
  (0, 3405)	1
  (0, 3654)	1
  (0, 3842)	4
  (0, 5221)	8
  (0, 6975)	3
  (0, 3452)	6
  (0, 5570)	6
  :	:
  (1032, 1819)	2
  (1032, 5155)	1
  (1032, 4233)	1
  (1032, 2488)	2
  (1032, 5606)	1
  (1032, 2747)	3
  (1032, 7259)	1
  (1032, 252)	4
  (1032, 5304)	2
  (1032, 339)	2
  (1032, 6710)	2
  (1032, 9365)	2
  (1032, 4555)	1
  (1032, 9404)	2
  (1032, 1025)	1
  (1032, 2582)	3
  (1032, 7108)	1
  (1032, 6657)	1
  (1032, 6754)	2
  (1032, 4554)	1
  (1032, 5237)	1
  (1032, 2227)	1
  (1032, 2422)	2
  (1032, 2001)	1
  (1032, 1365)	1


In [19]:
tfs = tfs.toarray().tolist()
counter_ = counter_.toarray()

In [20]:
doc_transactions = [[] for x in xrange(len(counter_))]
for i in xrange(counter_.shape[0]):
    for j in xrange(counter_.shape[1]):
        if counter_[i][j]:
            doc_transactions[i].append(j)
            
print doc_transactions

[[107, 582, 1580, 1618, 1959, 2294, 2340, 2412, 3405, 3452, 3458, 3654, 3842, 5221, 5263, 5570, 6319, 6975, 7834, 8494, 8548, 8568, 8569, 8865, 9868, 10190], [24, 107, 1005, 1304, 1365, 1376, 1741, 1868, 2001, 2186, 2227, 2422, 2664, 2959, 3190, 3452, 3491, 3585, 3586, 3593, 3594, 3790, 3945, 4145, 4306, 4321, 4554, 5221, 5237, 5310, 5371, 6266, 6269, 6307, 6796, 6899, 7121, 7673, 7680, 7692, 7837, 7905, 7923, 8022, 8129, 8736, 9341, 9363, 9942, 10172, 10236, 10287], [133, 135, 184, 340, 426, 1307, 1350, 1490, 1712, 1859, 1860, 2292, 2294, 2582, 3195, 3367, 3452, 3507, 3568, 4145, 4227, 4245, 4743, 4855, 4884, 5105, 5280, 5568, 5638, 5854, 6045, 6358, 6552, 6657, 6694, 6754, 6899, 7108, 7234, 7311, 7396, 7942, 7965, 8062, 8300, 9040, 9042, 9243, 9496, 9497, 9681], [66, 135, 1005, 1025, 1427, 1677, 1860, 1868, 1938, 2201, 2410, 2698, 3214, 3452, 3641, 3654, 4147, 4535, 4555, 4613, 4855, 5087, 5221, 5279, 5280, 5288, 5307, 5453, 5591, 5948, 6064, 6341, 6388, 6538, 6899, 6960, 6961, 8405,

In [26]:
import json
json.dump(doc_transactions,open('json_files/doc_transactions','w'))
json.dump(tfs, open('json_files/tfidf_matrix','w'))
json.dump(doc_tokens, open('json_files/doc_tokens','w'))
# json.dump(counter_.tolist(),open('index_terms','w'))